### Import Libraries

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "2,3"

from tqdm import tqdm_notebook as tqdm
from torch.autograd import Variable, Function
import time
import torch
import math
import numpy as np
import torchvision
import pandas as pd
import torch.nn as nn
import nibabel as nib
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.utils.data as data_utils
import torch.optim.lr_scheduler as schd  
import torchvision.transforms as transforms

torch.backends.cudnn.benchmark=True 
%matplotlib inline

### Parameter

In [78]:
batch_size = 100
elim_sparse_feature = True
cutoff = 0.2

### Preprocessing

In [108]:
raw_data = pd.read_csv("/home/emy24/tadpole/data/TADPOLE_InputData.csv", low_memory=False)
raw_data = raw_data.rename(columns = {'DX':'DX2'}) # Rename that column so there is no conflict in the algorithm

"""Obtain the IDs used for validation and test set"""
val_id = pd.read_csv("/home/emy24/tadpole/data/TADPOLE_TargetData_test.csv", low_memory=False)
test_id = pd.read_csv("/home/emy24/tadpole/data/TADPOLE_PredictTargetData_valid.csv", low_memory=False)

val_id = val_id["PTID_Key"].unique().astype("int")
test_id = test_id["PTID_Key"].unique().astype("int")

"""Columns 1888,1889, 1890 are numbers. Some number have inequality and some are wrongly typed as string
 The following lines convert everything to float and replace inqualities with NaN"""
raw_data.iloc[:,1888] = pd.to_numeric(raw_data.iloc[:,1888],errors = "coerce")
raw_data.iloc[:,1889] = pd.to_numeric(raw_data.iloc[:,1889],errors = "coerce")
raw_data.iloc[:,1890] = pd.to_numeric(raw_data.iloc[:,1890],errors = "coerce")
raw_data.iloc[:,3] = raw_data.iloc[:,3].astype('category')
raw_data.iloc[:,6] = raw_data.iloc[:,6].astype('category')
raw_data.iloc[:,10] = raw_data.iloc[:,10].astype('category')

""" Column 1891 is mixture of string and number. Get rid of the string """
raw_data.iloc[:,1891] = raw_data.iloc[:,1891].str.extract('(\d+)', expand = False)
raw_data.iloc[:,1891] = pd.to_numeric(raw_data.iloc[:,1891])

"""Remove Columns that does not need one-hot ecoding"""
n_row = raw_data.shape[0]
n_col = raw_data.shape[1]
removed_col = [] 
for n in range(n_col):
    if ("PTID_Key" in raw_data.columns[n] or
        "EXAMDATE" in raw_data.columns[n] or 
        "VERSION" in raw_data.columns[n] or 
        "update" in raw_data.columns[n] or 
        "RUNDATE" in raw_data.columns[n] or 
        "STATUS" in raw_data.columns[n] or 
        "BATCH_UPENNBIOMK9_04_19_17" in raw_data.columns[n] or 
        "KIT_UPENNBIOMK9_04_19_17" in raw_data.columns[n] or 
        "STDS_UPENNBIOMK9_04_19_17" in raw_data.columns[n]) :
        removed_col += [n]    

n = np.arange(n_col)
n = np.setxor1d(removed_col,n)
data = raw_data.iloc[:,n]

"""Search for categorical column and store where NaN are located"""
categorical_col = []
for c in range(data.shape[1]):
    if ((str(data.iloc[:,c].dtype)) == str("category") or 
        data.iloc[:,c].dtype is np.dtype('O')):
        categorical_col += [data.columns[c]]

"""One-hot encode"""
_nan_categories = data.isnull()
data = pd.get_dummies(data)

# Put NaN for the categorical data
for name in categorical_col:
    data.loc[_nan_categories[name], data.columns.str.startswith(name)] = np.NaN

"""Find the Location of NaN. It will be used for the indicator function"""
indicators = pd.isnull(data)
# 1 for existing data and 0 to nan
indicators = (indicators*1==0)*1

"""Reattach removed columns at the end"""
data = pd.concat([data, raw_data.iloc[:,removed_col]], axis=1)
indicators = pd.concat([indicators, raw_data["PTID_Key"]], axis=1) 
#Only ID is needed for the indicators

"""Replace -4 with NaN"""
data = data.replace(-4, np.NaN)
#Make it look pretty :) 
data = data.replace(np.nan,np.NaN)
data = data.replace(np.NAN,np.NaN)

"""Separate Train, Val, Test"""
groups = [data for _, data in data.groupby("PTID_Key")]
indicators_group = [indicators for _, indicators in indicators.groupby("PTID_Key")]

trn_indicator = []
val_indicator = []
test_indicator = []
xTrain = []
xVal = []
xTest = []
for n in range(len(groups)):
    subject = groups[n]["PTID_Key"].unique()[0].astype("int")
    if np.any(val_id == subject):
        xVal += [groups[n]]
        val_indicator += [indicators_group[n]]
    elif np.any(test_id == subject):
        xTest += [groups[n]]
        test_indicator += [indicators_group[n]]
    else:
        xTrain += [groups[n]]
        trn_indicator += [indicators_group[n]]
    
xTrain = pd.concat(xTrain).reset_index(drop=True)
xVal = pd.concat(xVal).reset_index(drop=True)
xTest = pd.concat(xTest).reset_index(drop=True)

trn_indicator = pd.concat(trn_indicator).reset_index(drop=True)
val_indicator = pd.concat(val_indicator).reset_index(drop=True)
test_indicator = pd.concat(test_indicator).reset_index(drop=True)

"""Standarize the Features. Only xTrain is used to calculate the mean
NaN replace by one."""
trn_mean = []
trn_std = []
xTrain_norm = xTrain
xVal_norm = xVal
xTest_norm = xTest
for c in range(1833):                   
    mean = xTrain.iloc[:,c].mean()
    std = xTrain.iloc[:,c].std()
    xTrain_norm.iloc[:,c] = (xTrain.iloc[:,c]-mean)/std
    xVal_norm.iloc[:,c] = (xVal.iloc[:,c]-mean)/std
    xTest_norm.iloc[:,c] = (xTest.iloc[:,c]-mean)/std
    trn_mean += [mean]
    trn_std += [std]

xTrain_norm.iloc[:,:1833] = xTrain_norm.iloc[:,:1833].replace(np.NaN, 0)
xVal_norm.iloc[:,:1833] = xVal_norm.iloc[:,:1833].replace(np.NaN, 0)
xTest_norm.iloc[:,:1833] = xTest_norm.iloc[:,:1833].replace(np.NaN, 0)

# For categorical data, NAN = 0
xTrain_norm.iloc[:,1833:] = xTrain_norm.iloc[:,1833:].replace(np.NaN, 0)
xVal_norm.iloc[:,1833:] = xVal_norm.iloc[:,1833:].replace(np.NaN, 0)
xTest_norm.iloc[:,1833:] = xTest_norm.iloc[:,1833:].replace(np.NaN, 0)

"""Note we need only the first 1943 colums
everything else is nonrelevant (dates,update,etc)"""
# The last column has the IDs
xTrain_norm = xTrain_norm.iloc[:,:1944]
xVal_norm = xVal_norm.iloc[:,:1944]
xTest_norm = xTest_norm.iloc[:,:1944]

trn_indicator = trn_indicator.iloc[:,:1944]
val_indicator = val_indicator.iloc[:,:1944]
test_indicator = test_indicator.iloc[:,:1944]

"""Eliminate Feature that few people have """
if elim_sparse_feature:
    percent_filled = np.sum(trn_indicator.as_matrix(), axis = 0)/trn_indicator.shape[0] 
    #percentage of the column that have data
    keep_col = percent_filled > cutoff
    keep_col_inx = np.argwhere(keep_col).flatten()

    xTrain_norm = xTrain_norm.iloc[:,keep_col_inx]
    xVal_norm = xVal_norm.iloc[:,keep_col_inx]
    xTest_norm = xTest_norm.iloc[:,keep_col_inx]

    trn_indicator = trn_indicator.iloc[:,keep_col_inx]
    val_indicator = val_indicator.iloc[:,keep_col_inx]
    test_indicator = test_indicator.iloc[:,keep_col_inx]
    


### Deep Autoencoder for Imputation

In [ ]:
"""Convert to numpy array. Note we need only the first 1943 colums
everything else is nonrelevant (dates,update,etc)"""
xTrain_norm = xTrain_norm.iloc[:,:1943].as_matrix()
xVal_norm = xVal_norm.iloc[:,:1943].as_matrix()
xTest_norm = xTest_norm.iloc[:,:1943].as_matrix()

trn_indicator = (trn_indicator.iloc[:,:1943].as_matrix()).astype("float")
val_indicator = (val_indicator.iloc[:,:1943].as_matrix()).astype("float")
test_indicator = (test_indicator.iloc[:,:1943].as_matrix()).astype("float")

"""Convert to Tensor"""
xTrain_norm = torch.from_numpy(xTrain_norm).float()
xVal_norm = torch.from_numpy(xVal_norm).float()
xTest_norm = torch.from_numpy(xTest_norm).float()

trn_indicator = torch.from_numpy(trn_indicator).float()
val_indicator = torch.from_numpy(val_indicator).float()
test_indicator = torch.from_numpy(test_indicator).float()

"""Dataloader"""
trainset = data_utils.TensorDataset(xTrain_norm, trn_indicator)
trainloader = torch.utils.data.DataLoader(trainset, batch_size= batch_size, 
                                          shuffle=True, num_workers= 2, drop_last=False)

valset = data_utils.TensorDataset(xVal_norm, val_indicator)
valloader = torch.utils.data.DataLoader(valset, batch_size = batch_size , 
                                         shuffle=False, num_workers= 2, drop_last=False)

testset = data_utils.TensorDataset(xTest_norm, test_indicator)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, 
                                        shuffle=False, num_workers= 2, drop_last=False)

"""Architecture"""
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()  
        self.dense = nn.Sequential(
            nn.Linear(in_features= 1943, out_features= 486),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(in_features= 486, out_features= 243),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(in_features= 243, out_features= 122),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(in_features= 122, out_features= 60),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(in_features= 60, out_features= 122),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(in_features= 122, out_features= 243),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(in_features= 243, out_features= 486),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(in_features= 486, out_features= 1943), 
        )
    def forward(self, x):
        out = self.dense(x)
        return out

autoencoder = Autoencoder()
autoencoder = torch.nn.DataParallel(autoencoder)
autoencoder.cuda() 

'''Optimazer'''
optimizer = optim.Adadelta(autoencoder.parameters())

'''Loss Functions'''
def adaptive_mse(reconstructed, target, weights):
    out = (reconstructed-target)**2
    out = out*weights
    loss = out.sum()
    return loss

"""Training"""
trn_loss = []
for epoch in tqdm(range(300)):  
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        autoencoder.train()

        inputs, indicator = data
        inputs = Variable(inputs).cuda()
        indicator = Variable(indicator).cuda()

        # Zero the parameter gradients 
        autoencoder.zero_grad()

        # forward + backward + optimize
        reconstruction = autoencoder(inputs)
        loss = adaptive_mse(reconstruction, inputs, indicator)
        loss.backward()
        optimizer.step()                 

        # Calculate statistics
        running_loss += loss.data[0]
    
    trn_loss += [running_loss/(i+1)]
    print('[Epoch %d] CAE loss: %.5f' 
          % (epoch + 1, running_loss/(i+1)))
    
    
#     #Validation
#     running_loss = 0.0
#     for i, data in enumerate(valloader, 0):       
#         autoencoder.eval()

#         inputs, labels = data
#         labels = labels.view(labels.numel()) #Format the labels the way pytorch wants it 
#         inputs = Variable(inputs).cuda()
#         labels = Variable(labels).cuda()

#         reconstruction = cae(inputs)
#         yPred = adv(cae(inputs))

#         # Calculate statistics
#         cae_running_loss += recons_function(reconstruction, inputs).data[0]
#         adv_running_loss += discrim_function(yPred, labels).data[0]
#         predicted = torch.round(yPred.data)
#         total += labels.size(0)
#         correct += (predicted.view(-1) == labels.data.view(-1)).sum()

#     adv_val_loss = adv_running_loss/(i+1)
#     cae_val_loss = cae_running_loss/(i+1)
#     adv_val_acc = (100*correct/total)

#     print('[Validation] Adv Acc: %.5f  Adv loss: %.5f  CAE loss: %.5f' 
#           % (adv_val_acc, adv_val_loss, cae_val_loss))    


In [ ]:
inputs

In [ ]:
reconstruction

In [ ]:
indicator

In [ ]:
# for i, data in enumerate(trainloader, 0):       
#     autoencoder.eval()

#     inputs, indicator = data
#     inputs = Variable(inputs).cuda()
#     reconstruction = autoencoder(inputs)

#     if i == 0:
#         target = inputs.data.cpu().numpy()
#         reconstructed = reconstruction.data.cpu().numpy()
#     else: 
#         labels = inputs.data.cpu().numpy()
#         decoded = reconstruction.data.cpu().numpy()
        
#         target = np.concatenate((target, labels), axis=0)
#         reconstructed = np.concatenate((reconstructed, decoded), axis=0)

indicator = pd.DataFrame(indicator.data.cpu().numpy())
reconstruction = pd.DataFrame(reconstruction.data.cpu().numpy())
inputs = pd.DataFrame(inputs.data.cpu().numpy())



In [ ]:
reconstruction

In [ ]:
inputs

In [ ]:
xTrain.iloc[:,:1943]